In [1]:
import ewtpy
#import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import antropy as ant
import mne
from mne.time_frequency import psd_array_welch
import yasa
from sklearn.feature_selection import SelectKBest, chi2
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mutual_info_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier

from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.problem_transform import ClassifierChain

from scipy.signal import butter,filtfilt

from scipy.stats import skew, kurtosis, entropy, pearsonr
from scipy.io import loadmat
from scipy import signal
import time


In [2]:
# Record the start time
start_time = time.time()

## Reading the data and creating labels

In [3]:
def ExtratData(mat_file_path,events_df):
    # Load the .mat file using scipy
    mat = loadmat(mat_file_path)

    # Extract EEG data
    eeg_data = mat['EEG'][0, 0]['data']
    
    # Create an MNE Info object, necessary to create RawArray
    info = mne.create_info(ch_names=['EEG1','EEG2'], sfreq=250, ch_types=['eeg','eeg'])

    # Create the RawArray object
    raw = mne.io.RawArray(eeg_data, info)

    # Now you can plot the raw data, create epochs, etc., using MNE functions.
    
    # Columns representing markers
    marker_columns = ['SS1', 'SS0', 'K1', 'K0', 'REM1', 'REM0', 'Son1', 'Son0', 'Soff1', 'Soff0', 'A1', 'A0', 'MS1', 'MS0']

    # 'Timestamp' is the column for event times
    timestamps = events_df['Timestamp'].values

    # 'Epochs' is the column for epoch number
    epoch_numbers = events_df['Epochs'].values

    # Create a binary column 'EventPresent' indicating the presence of any marker
    events_df['EventPresent'] = events_df[marker_columns].sum(axis=1).clip(upper=1)

    # Extract events where any marker is present
    event_times = timestamps[events_df['EventPresent'] == 1]
    epoch_ids = epoch_numbers[events_df['EventPresent'] == 1]

    # Create an events array for MNE
    events = np.column_stack([event_times, np.zeros(len(event_times), dtype=int), epoch_ids])

    # Define the sampling frequency of your raw EEG data
    sfreq = 250  # Replace with the actual sampling frequency

    # Define the epoch length
    tmin = 0  # Start of each epoch (at event)
    tmax = 30  # End of each epoch (30 seconds after event)


    # Create epochs
    epochs = mne.Epochs(raw, events=events, tmin=tmin, tmax=tmax, baseline=None, preload=True)

    data=epochs.get_data()
    drop_log=epochs.drop_log
    dropped_indices = [i for i, drop_status in enumerate(drop_log) if any(drop_status)]

    # Filter rows based on the index
    events_df_filtered = events_df.drop(dropped_indices, axis=0)

    # Resetting the index after dropping rows
    events_df_filtered.reset_index(drop=True, inplace=True)
    
    labels=events_df_filtered.values[:,2:-1]
    
    return data,labels

    

In [4]:
mat_file_path = 'train_S003_night5_hackathon_filt.mat'
events_df = pd.read_csv('train_S003_labeled.csv')

data1,label1=ExtratData(mat_file_path,events_df)

Creating RawArray with float64 data, n_channels=2, n_times=5772730
    Range : 0 ... 5772729 =      0.000 ... 23090.916 secs
Ready.
Not setting metadata
1050 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1050 events and 7501 original time points ...


C:\Users\Aashish Khilnani\AppData\Local\Temp\ipykernel_11252\3338525817.py:44: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(raw, events=events, tmin=tmin, tmax=tmax, baseline=None, preload=True)


1 bad epochs dropped


In [5]:
mat_file_path = 'train_S002_night1_hackathon_filt.mat'
events_df = pd.read_csv('train_S002_labeled.csv')

data2,label2=ExtratData(mat_file_path,events_df)

Creating RawArray with float64 data, n_channels=2, n_times=4965399
    Range : 0 ... 4965398 =      0.000 ... 19861.592 secs
Ready.
Not setting metadata
1191 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1191 events and 7501 original time points ...


C:\Users\Aashish Khilnani\AppData\Local\Temp\ipykernel_11252\3338525817.py:44: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(raw, events=events, tmin=tmin, tmax=tmax, baseline=None, preload=True)


0 bad epochs dropped


In [6]:
data1.shape, label1.shape, data2.shape, label2.shape

((1049, 2, 7501), (1049, 14), (1191, 2, 7501), (1191, 14))

## Concatenating the data

In [7]:
# Concatenate along the first axis
data_raw = np.concatenate((data1, data2), axis=0)
labels = np.concatenate((label1, label2), axis=0)

# Verify the shape
print(data_raw.shape, labels.shape) 

(2240, 2, 7501) (2240, 14)


## Applying the Filter

In [8]:
# Filter requirements.
T = 30.0         # Sample Period
fs = 250.0       # sample rate, Hz
cutoff = 2      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
nyq = 0.5 * fs  # Nyquist Frequency
order = 2       # sin wave can be approx represented as quadratic
n = 7501       #int(T * fs) # total number of samples

def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

In [9]:
data_filtered=data_raw
for epoch in range(data_raw.shape[0]):
    for singal_no in range(2):
        channel=data_raw[epoch,singal_no,:]
        data_filtered[epoch,singal_no,:]=butter_lowpass_filter(channel, cutoff, fs, order)
        

In [10]:
data_filtered.shape

(2240, 2, 7501)

## Adding Standard Scalar

In [11]:
scaler=MinMaxScaler()
DATA=data_filtered
for  epoch in range(data_filtered.shape[0]):
    channel=data_filtered[epoch]
    scaler.fit(channel.transpose())
    DATA[epoch]=np.array(scaler.transform(channel.transpose())).transpose()
DATA.shape

(2240, 2, 7501)

# Parameters for PSD

In [12]:
# Sampling frequency for Power Spectral Density (PSD)
fs = 250  # Hz

# Total duration of the signal
T = 30  # seconds

# Total number of samples in the signal
n = 7501


In [13]:
def extract_features_from_eeg(DATA, fs):
    """
    Extracts various statistical features from EEG signals.

    Parameters:
    - DATA: 3D numpy array representing EEG data (epochs x channels x samples)
    - fs: Sampling frequency of the EEG signals

    Returns:
    - feature_array: 2D numpy array containing extracted features for each epoch and channel
    """

    feature = []

    # Loop through each epoch
    for epoch_number in range(DATA.shape[0]):
        row = []

        # Loop through each channel (assuming there are 2 channels)
        for channel in range(2):
            f = DATA[epoch_number][channel, :]

            # Extract features using Empirical Wavelet Transform
            ewt, mfb, boundaries = ewtpy.EWT1D(f, N=3)
            for i in range(3):
                ewt_epoch = ewt[:, i]
                parameter = [
                    np.var(ewt_epoch),  # Variance
                    skew(ewt_epoch),  # Skewness
                    kurtosis(ewt_epoch),  # Kurtosis
                    np.ptp(ewt_epoch),  # Peak-to-Peak
                    np.sqrt(np.mean(ewt_epoch**2)),  # RMS (Root Mean Square)
                    np.std(ewt_epoch),  # Standard Deviation
                    ant.num_zerocross(ewt_epoch),  # Number of Zero Crossings
                    ant.hjorth_params(ewt_epoch)[0],  # Hjorth Mobility
                    ant.hjorth_params(ewt_epoch)[1],  # Hjorth Complexity
                    ant.petrosian_fd(ewt_epoch),  # Petrosian Fractal Dimension
                    ant.perm_entropy(ewt_epoch, normalize=True)  # Permutation Entropy
                ]
                row.extend(parameter)

            # Compute Power Spectral Density using Welch method
            frequencies, psd = signal.welch(f, fs=fs)
            parameter = [
                np.sum(psd),  # Sum of Power Spectral Density
                np.var(psd),  # Variance of Power Spectral Density
                skew(psd),  # Skewness of Power Spectral Density
                kurtosis(psd)  # Kurtosis of Power Spectral Density
            ]
            row.extend(parameter)

        # Compute Pearson correlation between the two EEG signals
        first_signal = DATA[epoch_number, 0, :]
        second_signal = DATA[epoch_number, 1, :]
        corr, _ = pearsonr(first_signal, second_signal)
        row.extend([corr])

        feature.append(row)

    # Convert the list of feature vectors into a numpy array
    feature_array = np.array(feature)

    return feature_array

# Example usage:
# Assuming you have EEG data in a variable named 'your_data' and sampling frequency 'your_sampling_frequency'
# features = extract_features_from_eeg(your_data, your_sampling_frequency)


In [14]:
feature_array=extract_features_from_eeg(DATA,fs)

In [15]:
feature_array.shape, labels.shape

((2240, 75), (2240, 14))

In [16]:
# Define feature names based on your extraction logic
feature_names = []

# Loop through each channel
for channel in range(2):
    for i in range(3):
        # Empirical Wavelet Transform (EWT) features
        feature_names.extend([
            f'Var_EWT_{channel + 1}_{i + 1}',
            f'Skew_EWT_{channel + 1}_{i + 1}',
            f'Kurtosis_EWT_{channel + 1}_{i + 1}',
            f'PTP_EWT_{channel + 1}_{i + 1}',
            f'RMS_EWT_{channel + 1}_{i + 1}',
            f'Std_EWT_{channel + 1}_{i + 1}',
            f'num_zerocross_{channel + 1}_{i + 1}',
            f'hjorth_Mobility_{channel + 1}_{i + 1}',
            f'hjorth_params_Complexity_{channel + 1}_{i + 1}',
            f'petrosian_fd_{channel + 1}_{i + 1}',
            f'perm_entropy_{channel + 1}_{i + 1}'
        ])

    # Power Spectral Density (PSD) features
    feature_names.extend([
        f'Sum_PSD_{channel + 1}',
        f'Var_PSD_{channel + 1}',
        f'Skew_PSD_{channel + 1}',
        f'Kurtosis_PSD_{channel + 1}',
    ])

# Add correlation feature
feature_names.extend(['Correlation'])

# Create a DataFrame with the feature names
feature_df = pd.DataFrame(feature_array, columns=feature_names)

# Display the DataFrame
feature_df


,Var_EWT_1_1,Skew_EWT_1_1,Kurtosis_EWT_1_1,PTP_EWT_1_1,RMS_EWT_1_1,Std_EWT_1_1,num_zerocross_1_1,hjorth_Mobility_1_1,hjorth_params_Complexity_1_1,petrosian_fd_1_1,...,num_zerocross_2_3,hjorth_Mobility_2_3,hjorth_params_Complexity_2_3,petrosian_fd_2_3,perm_entropy_2_3,Sum_PSD_2,Var_PSD_2,Skew_PSD_2,Kurtosis_PSD_2,Correlation
0,8.047852e-05,-2.778636,7.148752,0.045486,0.473971,0.008971,0.0,0.001784,1.431233,1.000042,...,195.0,0.083757,1.027202,1.001315,0.472223,0.006235,1.002098e-07,7.948852,66.869989,0.260266
1,4.664900e-05,2.595823,6.404240,0.034401,0.514982,0.006830,0.0,0.001824,1.378970,1.000030,...,195.0,0.080384,1.083967,1.001297,0.471174,0.004948,6.296145e-08,7.826115,64.452869,0.218227
2,1.022099e-04,-2.724840,6.913438,0.051068,0.509828,0.010110,0.0,0.001786,1.415778,1.000042,...,199.0,0.083048,1.061907,1.001309,0.471839,0.005144,7.135847e-08,8.176568,70.747248,0.216281
3,1.310649e-04,2.791871,7.129048,0.057462,0.530433,0.011448,0.0,0.001739,1.481271,1.000036,...,202.0,0.077492,1.140484,1.001285,0.470367,0.003270,2.653266e-08,8.017305,67.891591,0.390552
4,5.021182e-04,-2.751654,6.986143,0.112863,0.688225,0.022408,0.0,0.001765,1.431392,1.000042,...,203.0,0.088428,0.813573,1.001297,0.470891,0.000418,3.744857e-10,7.479498,59.269913,-0.011063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,2.877324e-05,2.731700,6.917791,0.027115,0.442916,0.005364,0.0,0.001768,1.427768,1.000036,...,206.0,0.081375,0.799646,1.001344,0.473818,0.000219,9.271348e-11,7.137775,53.293362,-0.023387
2236,6.659689e-07,0.824833,2.848043,0.004582,0.609177,0.000816,0.0,0.001586,1.688490,1.000036,...,201.0,0.086191,1.091511,1.001333,0.473022,0.006579,1.111299e-07,7.504096,56.598618,0.185708
2237,5.971271e-04,2.598614,6.361204,0.119624,0.460997,0.024436,0.0,0.001702,1.517970,1.000036,...,206.0,0.079289,1.000247,1.001315,0.472050,0.000592,7.245277e-10,7.177795,52.335145,0.029362
2238,2.176752e-07,-1.565303,1.167031,0.001773,0.454949,0.000467,0.0,0.001691,1.309025,1.000042,...,201.0,0.084194,0.926804,1.001315,0.472266,0.000583,7.552830e-10,7.354402,54.507287,0.113345


In [21]:
def frequency_features(DATA):
    feat=pd.DataFrame()
    for i in range(2):
        # Initialize an empty DataFrame
        bandpower_df = pd.DataFrame()
        bandpower_df1 = pd.DataFrame()

        # Loop through each signal in the first channel of DATA
        for sgnl in DATA[:, i, :]:
            # Calculate bandpower using the yasa library
            bp_df = yasa.bandpower(sgnl, sf=250)

            # Concatenate the bandpower DataFrame along columns
            bandpower_df = pd.concat([bandpower_df, bp_df], axis=0)

        # Remove the 'Gamma' column from the resulting DataFrame
        bandpower_df = bandpower_df.drop('Gamma', axis=1)

        # Reset the index and drop unnecessary columns
        bandpower_df = bandpower_df.iloc[:, :-2].reset_index(drop=True)

        # Create a new DataFrame for computed ratios and total power
        bandpower_df1[f'{i+1}_AT'] = bandpower_df['Alpha'] / bandpower_df['Theta']
        bandpower_df1[f'{i+1}_DB'] = bandpower_df['Delta'] / bandpower_df['Beta']
        bandpower_df1[f'{i+1}_DS'] = bandpower_df['Delta'] / bandpower_df['Sigma']
        bandpower_df1[f'{i+1}_DT'] = bandpower_df['Delta'] / bandpower_df['Theta']
        bandpower_df1[f'{i+1}_Total_Power'] = bandpower_df['TotalAbsPow']
        
        feat=pd.concat([feat, bandpower_df1], axis=1)
        # Display the resulting DataFrame
    return feat

In [24]:
feat=frequency_features(DATA)
feat

,1_AT,1_DB,1_DS,1_DT,1_Total_Power,2_AT,2_DB,2_DS,2_DT,2_Total_Power
0,0.084137,93340.574486,133116.787537,4852.783166,0.004387,0.008128,71168.989540,26267.177600,14.049083,0.005949
1,0.092951,56542.697117,81533.846307,4019.502585,0.003140,0.009297,62252.657783,24776.967363,12.833540,0.003958
2,0.106570,53495.783073,74681.759014,3466.881423,0.001510,0.012442,52155.700526,13106.232499,11.157713,0.003102
3,0.050132,97838.330227,143301.629604,3479.718406,0.001285,0.012785,121017.089279,22012.701518,14.496244,0.002529
4,0.043754,181619.967019,253883.787333,4424.299455,0.004256,0.011537,98977.414349,17997.430607,9.783482,0.000243
...,...,...,...,...,...,...,...,...,...,...
2235,0.138680,98629.465749,140371.556907,9104.641863,0.002354,0.004302,60479.108540,37453.544125,7.140565,0.000212
2236,0.033255,67958.639745,96164.434491,1360.208055,0.003511,0.004355,53597.691038,19696.454544,3.990534,0.005634
2237,0.038767,100682.076764,132303.086786,1976.065351,0.005286,0.007589,90516.260588,20499.941386,6.519749,0.000386
2238,0.050169,60574.148075,84021.874918,1936.483863,0.005156,0.006813,99934.558201,21347.843270,9.100218,0.000483


In [25]:
all_features_train=pd.concat([feature_df, feat], axis=1)
all_features_train

,Var_EWT_1_1,Skew_EWT_1_1,Kurtosis_EWT_1_1,PTP_EWT_1_1,RMS_EWT_1_1,Std_EWT_1_1,num_zerocross_1_1,hjorth_Mobility_1_1,hjorth_params_Complexity_1_1,petrosian_fd_1_1,...,1_AT,1_DB,1_DS,1_DT,1_Total_Power,2_AT,2_DB,2_DS,2_DT,2_Total_Power
0,8.047852e-05,-2.778636,7.148752,0.045486,0.473971,0.008971,0.0,0.001784,1.431233,1.000042,...,0.084137,93340.574486,133116.787537,4852.783166,0.004387,0.008128,71168.989540,26267.177600,14.049083,0.005949
1,4.664900e-05,2.595823,6.404240,0.034401,0.514982,0.006830,0.0,0.001824,1.378970,1.000030,...,0.092951,56542.697117,81533.846307,4019.502585,0.003140,0.009297,62252.657783,24776.967363,12.833540,0.003958
2,1.022099e-04,-2.724840,6.913438,0.051068,0.509828,0.010110,0.0,0.001786,1.415778,1.000042,...,0.106570,53495.783073,74681.759014,3466.881423,0.001510,0.012442,52155.700526,13106.232499,11.157713,0.003102
3,1.310649e-04,2.791871,7.129048,0.057462,0.530433,0.011448,0.0,0.001739,1.481271,1.000036,...,0.050132,97838.330227,143301.629604,3479.718406,0.001285,0.012785,121017.089279,22012.701518,14.496244,0.002529
4,5.021182e-04,-2.751654,6.986143,0.112863,0.688225,0.022408,0.0,0.001765,1.431392,1.000042,...,0.043754,181619.967019,253883.787333,4424.299455,0.004256,0.011537,98977.414349,17997.430607,9.783482,0.000243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,2.877324e-05,2.731700,6.917791,0.027115,0.442916,0.005364,0.0,0.001768,1.427768,1.000036,...,0.138680,98629.465749,140371.556907,9104.641863,0.002354,0.004302,60479.108540,37453.544125,7.140565,0.000212
2236,6.659689e-07,0.824833,2.848043,0.004582,0.609177,0.000816,0.0,0.001586,1.688490,1.000036,...,0.033255,67958.639745,96164.434491,1360.208055,0.003511,0.004355,53597.691038,19696.454544,3.990534,0.005634
2237,5.971271e-04,2.598614,6.361204,0.119624,0.460997,0.024436,0.0,0.001702,1.517970,1.000036,...,0.038767,100682.076764,132303.086786,1976.065351,0.005286,0.007589,90516.260588,20499.941386,6.519749,0.000386
2238,2.176752e-07,-1.565303,1.167031,0.001773,0.454949,0.000467,0.0,0.001691,1.309025,1.000042,...,0.050169,60574.148075,84021.874918,1936.483863,0.005156,0.006813,99934.558201,21347.843270,9.100218,0.000483


# Evaluation through Test Train Split

In [26]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features_train, labels, test_size=0.2, random_state=42)

# Initialize a multi-label classifier using LabelPowerset with a RandomForest base classifier
classifier = ClassifierChain(
    classifier=RandomForestClassifier(n_estimators=100, random_state=42),
    require_dense=[False, True]
)

# Train the classifier on the training set
classifier.fit(X_train, y_train)

# Predict labels on the test set
y_pred = classifier.predict(X_test)

# Calculate the F-Score using 'weighted' averaging for multi-label classification
f_score = f1_score(y_test, y_pred, average='weighted')

# Print the calculated F-Score
print("F-Score:", f_score)

F-Score: 0.629733204586962


C:\Users\Aashish Khilnani\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


## Now working with test data

In [27]:
mat_file_path = 'test_S004_night3_hackathon_filt.mat'
events_df = pd.read_csv('test_S004_unlabeled.csv')

In [29]:
# Load the .mat file using scipy
mat = loadmat(mat_file_path)
# Extract EEG data
eeg_data = mat['EEG'][0, 0]['data']
    
# Create an MNE Info object, necessary to create RawArray
info = mne.create_info(ch_names=['EEG1','EEG2'], sfreq=250, ch_types=['eeg','eeg'])

# Create the RawArray object
raw = mne.io.RawArray(eeg_data, info)

# Now you can plot the raw data, create epochs, etc., using MNE functions.
    
# Columns representing markers
marker_columns = ['SS1', 'SS0', 'K1', 'K0', 'REM1', 'REM0', 'Son1', 'Son0', 'Soff1', 'Soff0', 'A1', 'A0', 'MS1', 'MS0']

# 'Timestamp' is the column for event times
timestamps = events_df['Timestamp'].values

# 'Epochs' is the column for epoch number
epoch_numbers = events_df['Epoch'].values

# Create a binary column 'EventPresent' indicating the presence of any marker
events_df['EventPresent'] = events_df[marker_columns].sum(axis=1).clip(upper=1)+1

# Extract events where any marker is present
event_times = timestamps[events_df['EventPresent'] == 1]
epoch_ids = epoch_numbers[events_df['EventPresent'] == 1]

# Create an events array for MNE
events = np.column_stack([event_times, np.zeros(len(event_times), dtype=int), epoch_ids])

# Define the sampling frequency of your raw EEG data
sfreq = 250  # Replace with the actual sampling frequency

# Define the epoch length
tmin = 0  # Start of each epoch (at event)
tmax = 30  # End of each epoch (30 seconds after event)


# Create epochs
epochs = mne.Epochs(raw, events=events, tmin=tmin, tmax=tmax, baseline=None, preload=True)

test_data=epochs.get_data()

Creating RawArray with float64 data, n_channels=2, n_times=3600148
    Range : 0 ... 3600147 =      0.000 ... 14400.588 secs
Ready.
Not setting metadata
467 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 467 events and 7501 original time points ...
0 bad epochs dropped


C:\Users\Aashish Khilnani\AppData\Local\Temp\ipykernel_11252\2470799400.py:42: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(raw, events=events, tmin=tmin, tmax=tmax, baseline=None, preload=True)


## Applying Filter

In [31]:
test_data.shape

(467, 2, 7501)

In [ ]:
# Load the .mat file using scipy
mat = loadmat(mat_file_path)
# Extract EEG data
eeg_data = mat['EEG'][0, 0]['data']
    
# Create an MNE Info object, necessary to create RawArray
info = mne.create_info(ch_names=['EEG1','EEG2'], sfreq=250, ch_types=['eeg','eeg'])

# Create the RawArray object
raw = mne.io.RawArray(eeg_data, info)

# Now you can plot the raw data, create epochs, etc., using MNE functions.
    
# Columns representing markers
marker_columns = ['SS1', 'SS0', 'K1', 'K0', 'REM1', 'REM0', 'Son1', 'Son0', 'Soff1', 'Soff0', 'A1', 'A0', 'MS1', 'MS0']

# 'Timestamp' is the column for event times
timestamps = events_df['Timestamp'].values

# 'Epochs' is the column for epoch number
epoch_numbers = events_df['Epochs'].values

# Create a binary column 'EventPresent' indicating the presence of any marker
events_df['EventPresent'] = events_df[marker_columns].sum(axis=1).clip(upper=1)+1

# Extract events where any marker is present
event_times = timestamps[events_df['EventPresent'] == 1]
epoch_ids = epoch_numbers[events_df['EventPresent'] == 1]

# Create an events array for MNE
events = np.column_stack([event_times, np.zeros(len(event_times), dtype=int), epoch_ids])

# Define the sampling frequency of your raw EEG data
sfreq = 250  # Replace with the actual sampling frequency

# Define the epoch length
tmin = 0  # Start of each epoch (at event)
tmax = 30  # End of each epoch (30 seconds after event)


# Create epochs
epochs = mne.Epochs(raw, events=events, tmin=tmin, tmax=tmax, baseline=None, preload=True)

test_data=epochs.get_data()


    

In [32]:
testdata_filtered=test_data
for epoch in range(test_data.shape[0]):
    for singal_no in range(2):
        channel=test_data[epoch,singal_no,:]
        testdata_filtered[epoch,singal_no,:]=butter_lowpass_filter(channel, cutoff, fs, order)
        

In [33]:
testdata_filtered.shape

(467, 2, 7501)

In [34]:
scaler=MinMaxScaler()
test_DATA=testdata_filtered
for  epoch in range(testdata_filtered.shape[0]):
    channel=testdata_filtered[epoch]
    scaler.fit(channel.transpose())
    test_DATA[epoch]=np.array(scaler.transform(channel.transpose())).transpose()
test_DATA.shape

(467, 2, 7501)

In [37]:
feature_array_test=extract_features_from_eeg(test_DATA,fs)

In [38]:
# Create a DataFrame with the feature names
feature_test_df = pd.DataFrame(feature_array_test, columns=feature_names)

# Display the DataFrame
feature_test_df


,Var_EWT_1_1,Skew_EWT_1_1,Kurtosis_EWT_1_1,PTP_EWT_1_1,RMS_EWT_1_1,Std_EWT_1_1,num_zerocross_1_1,hjorth_Mobility_1_1,hjorth_params_Complexity_1_1,petrosian_fd_1_1,...,num_zerocross_2_3,hjorth_Mobility_2_3,hjorth_params_Complexity_2_3,petrosian_fd_2_3,perm_entropy_2_3,Sum_PSD_2,Var_PSD_2,Skew_PSD_2,Kurtosis_PSD_2,Correlation
0,3.020704e-05,-1.630374,1.449547,0.021515,0.513656,0.005496,0.0,0.001899,1.275801,1.000042,...,209.0,0.083702,1.075837,1.001439,0.478821,0.004695,5.299397e-08,7.813464,63.932628,0.349648
1,2.701310e-07,1.593295,3.862049,0.002813,0.515874,0.000520,0.0,0.001460,1.730352,1.000042,...,209.0,0.085916,1.054107,1.001415,0.477596,0.005235,6.657769e-08,7.814367,63.794672,0.348215
2,4.084247e-05,-2.801143,7.126541,0.031970,0.514632,0.006391,0.0,0.001727,1.474943,1.000042,...,214.0,0.081082,0.924622,1.001421,0.477944,0.000534,5.485149e-10,7.030199,51.197503,0.247729
3,4.033686e-05,2.443227,5.613991,0.031123,0.491417,0.006351,0.0,0.001800,1.365794,1.000030,...,211.0,0.088404,1.097455,1.001421,0.477861,0.005194,6.395647e-08,7.393934,54.949008,0.340352
4,5.206729e-05,-2.373724,5.330260,0.035341,0.487132,0.007216,0.0,0.001830,1.350516,1.000030,...,205.0,0.081923,0.956446,1.001380,0.475485,0.000604,6.921630e-10,6.982784,50.492982,0.204297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,2.130369e-04,2.583265,6.296674,0.072857,0.438039,0.014596,0.0,0.001816,1.383641,1.000042,...,214.0,0.088429,1.097176,1.001344,0.473842,0.002509,1.287149e-08,6.973687,50.088524,0.247054
463,4.444447e-04,2.426844,5.619362,0.104357,0.460286,0.021082,0.0,0.001828,1.354813,1.000030,...,223.0,0.089040,1.045185,1.001380,0.475809,0.001331,3.555390e-09,6.894475,48.353883,0.190513
464,1.402479e-04,-2.779045,7.091551,0.059731,0.544070,0.011843,0.0,0.001756,1.449969,1.000030,...,217.0,0.086763,1.092649,1.001475,0.480670,0.001430,4.456938e-09,7.234743,52.456011,0.253191
465,1.144091e-05,-1.789242,2.332766,0.014683,0.513611,0.003382,0.0,0.001920,1.281510,1.000042,...,219.0,0.096690,1.127736,1.001528,0.483418,0.013986,4.257784e-07,7.031723,49.904801,0.220767


In [39]:
feat_test=frequency_features(test_DATA)
feat_test

,1_AT,1_DB,1_DS,1_DT,1_Total_Power,2_AT,2_DB,2_DS,2_DT,2_Total_Power
0,0.050591,578857.906681,815245.118043,16990.884036,0.015683,0.010629,136766.300626,28434.935639,7.773074,0.003135
1,0.131767,97933.207752,136863.411669,8110.888147,0.008899,0.010485,69111.007175,25009.044351,7.905886,0.003386
2,0.108958,125130.039944,174761.788942,8857.127019,0.006911,0.009416,41864.835475,14109.218753,4.646637,0.000268
3,0.108329,61460.876210,88506.170747,4522.738186,0.004794,0.010972,83600.517817,23900.032560,8.141538,0.004493
4,0.074782,85055.466926,117711.211012,4118.150813,0.004616,0.005520,38525.598705,16615.949507,5.085064,0.000437
...,...,...,...,...,...,...,...,...,...,...
462,0.030381,103549.545390,148189.221082,1782.643822,0.007252,0.005070,71887.800312,25896.990820,3.644625,0.002130
463,0.045696,55865.249168,78507.310062,1812.221626,0.014984,0.005185,61438.817540,29094.943655,3.460937,0.001233
464,0.042974,74825.830007,106193.804160,1883.604137,0.007244,0.005897,78439.950056,46098.019054,4.174357,0.001056
465,0.021199,217212.558978,299729.018875,1474.205128,0.005939,0.006394,161067.887461,50993.938565,3.693078,0.013646


In [40]:
all_features_test=pd.concat([feature_test_df, feat_test], axis=1)
all_features_test

,Var_EWT_1_1,Skew_EWT_1_1,Kurtosis_EWT_1_1,PTP_EWT_1_1,RMS_EWT_1_1,Std_EWT_1_1,num_zerocross_1_1,hjorth_Mobility_1_1,hjorth_params_Complexity_1_1,petrosian_fd_1_1,...,1_AT,1_DB,1_DS,1_DT,1_Total_Power,2_AT,2_DB,2_DS,2_DT,2_Total_Power
0,3.020704e-05,-1.630374,1.449547,0.021515,0.513656,0.005496,0.0,0.001899,1.275801,1.000042,...,0.050591,578857.906681,815245.118043,16990.884036,0.015683,0.010629,136766.300626,28434.935639,7.773074,0.003135
1,2.701310e-07,1.593295,3.862049,0.002813,0.515874,0.000520,0.0,0.001460,1.730352,1.000042,...,0.131767,97933.207752,136863.411669,8110.888147,0.008899,0.010485,69111.007175,25009.044351,7.905886,0.003386
2,4.084247e-05,-2.801143,7.126541,0.031970,0.514632,0.006391,0.0,0.001727,1.474943,1.000042,...,0.108958,125130.039944,174761.788942,8857.127019,0.006911,0.009416,41864.835475,14109.218753,4.646637,0.000268
3,4.033686e-05,2.443227,5.613991,0.031123,0.491417,0.006351,0.0,0.001800,1.365794,1.000030,...,0.108329,61460.876210,88506.170747,4522.738186,0.004794,0.010972,83600.517817,23900.032560,8.141538,0.004493
4,5.206729e-05,-2.373724,5.330260,0.035341,0.487132,0.007216,0.0,0.001830,1.350516,1.000030,...,0.074782,85055.466926,117711.211012,4118.150813,0.004616,0.005520,38525.598705,16615.949507,5.085064,0.000437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,2.130369e-04,2.583265,6.296674,0.072857,0.438039,0.014596,0.0,0.001816,1.383641,1.000042,...,0.030381,103549.545390,148189.221082,1782.643822,0.007252,0.005070,71887.800312,25896.990820,3.644625,0.002130
463,4.444447e-04,2.426844,5.619362,0.104357,0.460286,0.021082,0.0,0.001828,1.354813,1.000030,...,0.045696,55865.249168,78507.310062,1812.221626,0.014984,0.005185,61438.817540,29094.943655,3.460937,0.001233
464,1.402479e-04,-2.779045,7.091551,0.059731,0.544070,0.011843,0.0,0.001756,1.449969,1.000030,...,0.042974,74825.830007,106193.804160,1883.604137,0.007244,0.005897,78439.950056,46098.019054,4.174357,0.001056
465,1.144091e-05,-1.789242,2.332766,0.014683,0.513611,0.003382,0.0,0.001920,1.281510,1.000042,...,0.021199,217212.558978,299729.018875,1474.205128,0.005939,0.006394,161067.887461,50993.938565,3.693078,0.013646


In [48]:

# initialize LabelPowerset multi-label classifier with a RandomForest
classifier = ClassifierChain(
    classifier = RandomForestClassifier(n_estimators=100, random_state=42),
    require_dense = [False, True]
)

# train
classifier.fit(all_features_train, labels)

# Predict on the test set
y_pred_dense = classifier.predict(all_features_test)

y_pred_dense

<467x14 sparse matrix of type '<class 'numpy.float64'>'
	with 614 stored elements in Compressed Sparse Column format>

In [49]:
y_pred = y_pred_dense.toarray().astype(int)
y_pred,y_pred.shape

(array([[0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]]),
 (467, 14))

In [46]:
events_df.columns

Index(['Timestamp', 'Epoch', 'SS1', 'SS0', 'K1', 'K0', 'REM1', 'REM0', 'Son1',
       'Son0', 'Soff1', 'Soff0', 'A1', 'A0', 'MS1', 'MS0', 'EventPresent'],
      dtype='object')

In [50]:
events_df.iloc[:,2:-1]=y_pred
events_df

,Timestamp,Epoch,SS1,SS0,K1,K0,REM1,REM0,Son1,Son0,Soff1,Soff0,A1,A0,MS1,MS0,EventPresent
0,10483,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0
1,11543,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0
2,12792,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0
3,14027,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1.0
4,15239,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,3041643,406,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0
463,3043473,406,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0
464,3069693,410,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1.0
465,3071225,410,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0


In [51]:
events_df = events_df.drop(columns='EventPresent')


In [52]:
events_df

,Timestamp,Epoch,SS1,SS0,K1,K0,REM1,REM0,Son1,Son0,Soff1,Soff0,A1,A0,MS1,MS0
0,10483,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,11543,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,12792,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,14027,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0
4,15239,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,3041643,406,0,1,0,0,0,0,0,0,0,0,0,0,0,0
463,3043473,406,0,1,0,0,0,0,0,0,0,0,0,0,0,0
464,3069693,410,0,1,0,0,1,0,0,0,0,0,0,0,0,0
465,3071225,410,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
# Assuming you want to save the DataFrame to a file named 'events_df.csv'
events_df.to_csv('events_df.csv', index=False)
